In [1]:
%reset -f

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.signal as signal
import matplotlib.gridspec as gridspec
import time

from scipy.signal import butter, filtfilt
from matplotlib.patches import ConnectionPatch
from MINE.Log import Log
from MINE.Analysis import SessionAnalytics, ExperimentAnalytics
from MINE.StreamFilter import IStreamFilter, TimestampStreamFilter
from MINE.SessionFilters import ISessionFilter, ContainsStreamSessionFilter, ContainsMarkersSessionFilter
from MINE.StreamProcessing import StreamProcesses, PPG_Event
from MINE.StreamOutput import StreamOutput

In [3]:
_session_analytics: SessionAnalytics = SessionAnalytics.create_from_path("V:/Data/Raw/Emotibit Test/sub-P001_ses-S001_task-Default_run-001_eeg.xdf")
_session_analytics.localise_timestamps()

In [4]:
start = time.time()
StreamProcesses.butterworth_filter(_session_analytics, "PPG_GRN", "PPG_GRN_Filtered")
Log.message(f"1: Time taken: {time.time() - start:.3f} seconds")

start = time.time()
StreamProcesses.detect_ppg_peaks(_session_analytics, "PPG_GRN_Filtered", "PPG_GRN_Filtered_Peak_Annotations")
Log.message(f"2: Time taken: {time.time() - start} seconds")

start = time.time()
StreamProcesses.create_vectors_from_component_streams(_session_analytics, "ACC_X", "ACC_Y", "ACC_Z", "ACC_Vector")
Log.message(f"3: Time taken: {time.time() - start} seconds")

start = time.time()
StreamProcesses.calculate_magnitudes_from_vector_stream(_session_analytics, "ACC_Vector", "ACC_Magnitude")
Log.message(f"4: Time taken: {time.time() - start} seconds")

[ Message ] 1: Time taken: 1.335 seconds
[ Message ] 2: Time taken: 0.10969066619873047 seconds
[ Message ] 3: Time taken: 0.01764082908630371 seconds
[ Message ] 4: Time taken: 0.025355815887451172 seconds


In [ ]:
def plot_filtered_ppg_subfigure(session_analytics: SessionAnalytics, axis: plt.Axes):
    ppg_filtered_stream: pd.DataFrame = session_analytics.stream_data_dictionary["PPG_GRN_Filtered"]
    ppg_filtered_peak_annotations = session_analytics.stream_data_dictionary["PPG_GRN_Filtered_Peak_Annotations"]

    axis.plot(ppg_filtered_stream["Timestamp"], ppg_filtered_stream["Value"])

    for _, row in ppg_filtered_peak_annotations.iterrows():
        timestamp: float = row["Timestamp"]
        ppg_event: PPG_Event = row["Value"]

        axis.scatter(timestamp, ppg_event.systolic_peak_value, c = "orange")

        if ppg_event.diastolic_trough_timestamp_offset and ppg_event.diastolic_trough_value:
            axis.scatter(timestamp + ppg_event.diastolic_trough_timestamp_offset, ppg_event.diastolic_trough_value, c = "blue")

        if ppg_event.diastolic_peak_timestamp_offset and ppg_event.diastolic_peak_value:
            axis.scatter(timestamp + ppg_event.diastolic_peak_timestamp_offset, ppg_event.diastolic_peak_value, c = "orange", marker = "x")

        if ppg_event.dicrotic_notch_timestamp_offset and ppg_event.dicrotic_notch_value:
            axis.scatter(timestamp + ppg_event.dicrotic_notch_timestamp_offset, ppg_event.dicrotic_notch_value, c = "blue", marker = "x")


def plot_heartrate_subfigure(session_analytics: SessionAnalytics, axis: plt.Axes, sample_duration: float = 20, sampling_step_count: float = 5):
    ppg_filtered_stream: pd.DataFrame = session_analytics.stream_data_dictionary["PPG_GRN_Filtered"]
    systolic_peaks = session_analytics.stream_data_dictionary["PPG_GRN_Filtered_Peak_Annotations"]
    duration: float = ppg_filtered_stream["Timestamp"].iloc[-1] - ppg_filtered_stream["Timestamp"].iloc[0]

    #region [ Calculate Average Heartrate ]
    bpm_dataframe = pd.DataFrame(columns=["Timestamp", "BPM"])

    min_bpm: float = float("inf")
    max_bpm: float = 0

    sample_time = -60 + (sample_duration / 2)
    while sample_time < (duration - (sample_duration / 2)):
        peaks_subset = systolic_peaks[
            (systolic_peaks["Timestamp"] > (sample_time - (sample_duration / 2))) &
            (systolic_peaks["Timestamp"] < (sample_time + (sample_duration / 2)))
        ]

        bpm = len(peaks_subset) * (60 / sample_duration)

        if bpm < min_bpm: min_bpm = bpm
        if bpm > max_bpm: max_bpm = bpm

        bpm_dataframe.loc[len(bpm_dataframe)] = [
            sample_time,
            bpm
        ]

        sample_time = sample_time + sampling_step_count
    #endregion

    plotted_data = bpm_dataframe.loc[bpm_dataframe["Timestamp"] > 20]

    markers = session_analytics.stream_data_dictionary["FC_Markers"]
    sample_start: float = 0
    for _, row in markers.iterrows():
        sample_end = row["Timestamp"]

        period_sample = plotted_data.loc[
            (plotted_data["Timestamp"] > sample_start) &
            (plotted_data["Timestamp"] < sample_end)
        ]

        period_average = period_sample["BPM"].mean()
        axis.hlines(y=period_average, xmin=sample_start, xmax=sample_end, color="red", alpha=0.5)

        sample_start = sample_end

    axis.plot(plotted_data["Timestamp"], plotted_data["BPM"])
    axis.set_ylim(50, 100)

def plot_accelerometer_magnitude_subfigure(session_analytics: SessionAnalytics, axis: plt.Axes):
    accelerometer_magnitude_stream: pd.DataFrame = session_analytics.stream_data_dictionary["ACC_Magnitude"]
    accelerometer_x: pd.DataFrame = session_analytics.stream_data_dictionary["ACC_X"]
    accelerometer_y: pd.DataFrame = session_analytics.stream_data_dictionary["ACC_Y"]
    accelerometer_z: pd.DataFrame = session_analytics.stream_data_dictionary["ACC_Z"]

    axis.plot(accelerometer_magnitude_stream["Timestamp"], accelerometer_magnitude_stream["Value"], c = "black")
    axis.plot(accelerometer_x["Timestamp"], accelerometer_x["Value"], alpha=0.5, linestyle="dashed", c = "red")
    axis.plot(accelerometer_y["Timestamp"], accelerometer_y["Value"], alpha=0.5, linestyle="dashed", c = "green")
    axis.plot(accelerometer_z["Timestamp"], accelerometer_z["Value"], alpha=0.5, linestyle="dashed", c = "blue")

def plot_markers_on_all_subfigures(session_analytics: SessionAnalytics, axes: list[plt.Axes]):
    markers: pd.DataFrame = session_analytics.stream_data_dictionary["FC_Markers"]
    for _, row in markers.iterrows():
        for axis in axes: axis.vlines(row["Timestamp"], ymin=axis.get_ylim()[0], ymax=axis.get_ylim()[1], color="red", linestyles="dashed", alpha=0.5)
        for axis in axes: axis.text(x=row["Timestamp"], y=axis.get_ylim()[0], s=row["Value"], ha='center', va='center')
    pass

def plot_experiment_data(session_analytics: SessionAnalytics):
    ppg_filtered_stream: pd.DataFrame = session_analytics.stream_data_dictionary["PPG_GRN_Filtered"]
    duration: float = ppg_filtered_stream["Timestamp"].iloc[-1] - ppg_filtered_stream["Timestamp"].iloc[0]

    sub_figures: int = 3
    figure, axes = plt.subplots(nrows=sub_figures, ncols=1, figsize=(duration * 0.5, sub_figures * 4), dpi=300)

    plot_filtered_ppg_subfigure(_session_analytics, axes[0])
    #plot_heartrate_subfigure(_session_analytics, axes[1])
    #plot_accelerometer_magnitude_subfigure(_session_analytics, axes[2])
    #plot_markers_on_all_subfigures(_session_analytics, axes)

    for axis in axes: axis.set_xlim(0, duration)

    plt.title("Freds Data")
    plt.savefig(f"V:/Exports/Freds Data.png", dpi=300)
    plt.close()

plot_experiment_data(_session_analytics)